# Peform statistical analyses of GNSS station locations and tropospheric zenith delays

**Author**: Simran Sangha, David Bekaert - Jet Propulsion Laboratory

This notebook provides an overview of the functionality included in the **`raiderStats.py`** program. Specifically, we outline examples on how to perform basic statistical analyses of GNSS station location and tropospheric zenith delay information over a user defined area of interest, span of time, and seasonal interval. In this notebook, we query GNSS stations spanning California between 2016 and 2018. 

We will outline the following statistical analysis and filtering options:
- Restrict analyses to range of years
- Restrict analyses to range of months (i.e. seasonal interval)
- Illustrate station distribution and tropospheric zenith delay mean/standard deviation
- Illustrate gridded distribution and tropospheric zenith delay mean/standard deviation
- Generate variogram plots across specified time periods
- Perform basic seasonal amplitude/phase analyses
- Examine residuals between weather-models and collocated GNSS stations

<div class="alert alert-warning">
Both the initial setup (<b>Prep A</b> section) and download of the data (<b>Prep B</b> section) should be run at the start of the notebook. All other sections (examples and applications) do not need to be run in order.
</div>

<div class="alert alert-danger">
<b>Potential Errors/Warnings:</b> 
    
- RAiDER needs to be installed to run this notebook
- Most of the examples below specify the use of all available cpus `--cpus all`. If you anticipate this would be an issue on your machine, please adjust to a more conservative number accordingly (e.g. `--cpus 8`).
- Just note that downloading data with less than 8 cpus may be very time-consuming and not recommended, while most other steps should run quickly.
</div>

<div class="alert alert-info">
    <b>Terminology:</b>
    
- *GNSS*: Stands for Global Navigation Satellite System. Describes any satellite constellation providing global or regional positioning, navigation, and timing services.
- *tropospheric zenith delay*: The precise atmospheric delay satellite signals experience when propagating through the troposphere.
- *variogram*: Characterization of the difference between field values at two locations.
- *empirical variogram / semivariance*: Provides a description of how the data are correlated with distance. 1/2 * sqrt(obs1 - obs2)
- *experimental variogram*: A discrete function calculated using a measure of variability between pairs of points at various distances.
- *sill/variance*: The limit of the variogram representing where the variogram appears to level off, tending to infinity lag distances.
- *range*: The distance in which the difference of the variogram from the sill becomes negligible, such that the data are no longer autocorrelated.
    </div>
    

## Table of Contents:
<a id='example_TOC'></a>

[**Overview of the raiderStats.py program**](#overview)
- [1. Basic user input options](#overview_1)
- [2. Run parameters](#overview_2)
- [3. Optional controls for spatiotemporal subsetting](#overview_3)
- [4. Supported types of individual station scatter-plots](#overview_4)
- [5. Supported types of gridded station plots](#overview_5)
- [6. Supported types of variogram plots](#overview_6)
- [7. Optional controls for plotting](#overview_7)

[**Examples of the raiderStats.py program**](#examples)
- [Example 1. Generate all individual station scatter-plots, as listed under section #4](#example_1)
- [Example 2. Generate all basic gridded station plots, as listed under section #5](#example_2)
  - [Example 2a. Redo plots efficiently using generated grid raster files](#example_2a)
- [Example 3. Generate gridded mean tropospheric zenith delay plot, with stations superimposed](#example_3)
- [Example 4. Generate variogram plots](#example_4)
- [Example 5. Generate seasonal phase/amplitude plots](#example_5)
- [Example 6. Generate weather model/GNSS residual plots](#example_6)

## Prep A: Initial setup of the notebook

Below we set up the directory structure for this notebook exercise. In addition, we load the required modules into our python environment using the **`import`** command.

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt

## Defining the home and data directories
tutorial_home_dir = os.path.abspath(os.getcwd())
work_dir = os.path.abspath(os.getcwd())
print("Tutorial directory: ", tutorial_home_dir)
print("Work directory: ", work_dir)

# Verifying if RAiDER is installed correctly
try:
    from RAiDER.cli import statsPlot
except:
    raise Exception('RAiDER is missing from your PYTHONPATH')

os.chdir(work_dir)

Tutorial directory:  /Users/ssangha/Downloads/snap_setup/stable_oct5_2021/RAiDER/RAiDER-docs/notebooks/raiderStats
Work directory:  /Users/ssangha/Downloads/snap_setup/stable_oct5_2021/RAiDER/RAiDER-docs/notebooks/raiderStats


## Prep B: Download prerequisite GNSS station location and tropospheric zenith delay information with the **`raiderDownloadGNSS.py`** program

Virtually access GNSS station location and zenith delay information for the years '2016-2018', for every 5 days, at a UTC time of day 'HH:MM:SS' of '00:00:00', and across a geographic bounding box '32 42 -125 -114' spanning over California.

The footprint of the specified geographic bounding box is again depicted in **Fig. 1**.

In addition to querying for multiple years, we will also experiment with using the maximum number of allowed CPUs to save some time! Recall again that the default number of CPUs used for parallelization is 8.

Note these features and similar examples are outlined in more detail in the companion notebook **`raiderDownloadGNSS/raiderDownloadGNSS_tutorial.ipynb`**. The outputs of the last example for that notebook will be used for exercises 1-5. So if this example has already been executed, the following prep sets a symlink pointing to these outputs over from the previous notebook to avoid duplicating efforts and wasting time/coffee.

In [5]:
## Check if outputs from the relevant raiderDownloadGNSS notebook exercise exist
## Only run the raiderDownloadGNSS program if these outputs don't exist
gnss_tutorial_home_dir = '../raiderDownloadGNSS/products_ex2/UNRcombinedGPS_ztd.csv'
run_command = "raiderDownloadGNSS.py --out products --date 20160101 20181231 5 --returntime '00:00:00' \
              --bounding_box '32 42 -125 -114' --cpus all"
if not os.path.exists(gnss_tutorial_home_dir):
    from RAiDER.cli import statsPlot
    print('Download data needed for this exercise with the following command: {}'.format(run_command))
    os.system(run_command)
else:
    os.symlink(os.path.dirname(gnss_tutorial_home_dir), 'products')

Download data needed for this exercise with the following command: raiderDownloadGNSS.py --out products_ex2 --date 20160101 20181231 5 --returntime '00:00:00'               --bounding_box '32 42 -125 -114' --cpus all
1781 stations were found in [32.0, 42.0, 235.0, 246.0]
Currently checking station 34A1 in 2016
Currently checking station BAMO in 2018
Currently checking station BRD1 in 2017
Currently checking station CAES in 2016
Currently checking station CAS4 in 2018
Currently checking station CLAR in 2017
Currently checking station CYTE in 2016
Currently checking station EDNA in 2018
Currently checking station 34A1 in 2017
Currently checking station BRD1 in 2018
Currently checking station CAES in 2017
Currently checking station BAR1 in 2016
Currently checking station EDPP in 2016
Currently checking station CYTE in 2017
Currently checking station CLAR in 2018
Currently checking station CAS6 in 2016
Currently checking station CAES in 2018
Currently checking station 34A1 in 2018
Currentl

All of the extracted tropospheric zenith delay information stored under **`GPS_delays`** is concatenated with the GNSS station location information stored under **`gnssStationList_overbbox.csv`** into a primary comprehensive file **`UNRcombinedGPS_ztd.csv`**

**`UNRcombinedGPS_ztd.csv`** may in turn be directly used to perform basic statistical analyses using **`raiderStats.py`**.

<img src="support_docs/bbox_footprint.png" alt="footprint" width="700">
<center><b>Fig. 1</b> Footprint of geopraphic bounding box used in the examples below. </center>

## Overview of the raiderStats.py program
<a id='overview'></a>

The **`raiderStats.py`** program provides a suite of convinient statistical analyses of GNSS station locations and tropospheric zenith delays.

Running **`raiderStats.py`** with the **`-h`** option will show the parameter options and outline several basic, practical examples. 

Let us explore these options:

In [2]:
!raiderStats.py -h

usage: raiderStats.py [-h] -f FNAME [-c COL_NAME] [-u UNIT] [-w WORKDIR]
                      [--cpus CPUS] [-verbose] [-b BOUNDING_BOX] [-sp SPACING]
                      [-ti TIMEINTERVAL] [-si SEASONALINTERVAL]
                      [-oe OBS_ERRLIMIT] [-figdpi FIGDPI] [-title USER_TITLE]
                      [-fmt PLOT_FMT] [-cb CBOUNDS]
                      [-cp COLORPERCENTILE COLORPERCENTILE] [-cm USR_COLORMAP]
                      [-dt DENSITYTHRESHOLD] [-sg] [-dg] [-tl] [-plotall]
                      [-min_span MIN_SPAN MIN_SPAN]
                      [-period_limit PERIOD_LIMIT] [-station_distribution]
                      [-station_delay_mean] [-station_delay_median]
                      [-station_delay_stdev] [-station_seasonal_phase]
                      [-phaseamp_per_station] [-grid_heatmap]
                      [-grid_delay_mean] [-grid_delay_median]
                      [-grid_delay_stdev] [-grid_seasonal_phase]
                      [-grid_delay_absolute_me

### 1. Basic user input options
<a id='overview_1'></a>

#### Input CSV file (**`--file FNAME`**)

**REQUIRED** argument. Provide a valid CSV file as input through **`--file`** which lists the GNSS station IDs (ID), lat/lon coordinates (Lat,Lon), dates in YYYY-MM-DD format (Date), and the desired data field in units of meters.

Note that the complementary **`raiderDownloadGNSS.py`** format generates such a primary CSV file named **`UNRcombinedGPS_ztd.csv`** that contains all such fields and is already formatted as expected by **`raiderStats.py`**. Please refer to the accompanying **`raiderDownloadGNSS/raiderDownloadGNSS_tutorial.ipynb `** for more details and practical examples.

Additionally supported are rasters containing gridded arrays generated with a previous `raiderStats.py` call with the `--grid_to_raster` option specified. This is designed to be a quick means to replot previously generated arrays while circumventing the overhead involved with the initial gridding/processing of the original CSV file.
E.g. if specified with **`--grid_delay_mean`**, then a raster file named `ZTD_grid_delay_mean.tif` containing the gridded mean delay will be generated


#### Data column name (**`--column_name COL_NAME`**)

Specify name of data column in input CSV file through **`--column_name `** that you wish to perform statistical analyses on. Input assumed to be in units of meters.

Default input column name set to **`ZTD`**, the name assigned to tropospheric zenith delays populated under the **`ZTDCombinedGPS_ztd.csv`** file generated through **`raiderDownloadGNSS.py`**

The column name is always prepended to output products (e.g. `ZTD_grid_heatmap.tif` and `ZTD_grid_heatmap.png`)

#### Data column unit (**`--unit UNIT`**)

Specify unit for output rasters/graphics through **`--unit`**. Again, input assumed to be in units of meters so it will be converted into meters if not already in meters.

### 2. Run parameters
<a id='overview_2'></a>

#### Output directory (**`--workdir WORKDIR`**)

Specify directory to deposit all outputs into with **`--workdir`**. Absolute and relative paths are both supported.

By default, outputs will be deposited into the current working directory where the program is launched.

#### Number of CPUs to be used (**`--cpus NUMCPUS`**)

Specify number of cpus to be used for multiprocessing with **`--cpus`**. For most cases, multiprocessing is essential in order to access data and perform statistical analyses within a reasonable amount of time.

May specify **`--cpus all`** at your own discretion in order to leverage all available CPUs on your system.

By default 8 CPUs will be used.

#### Verbose mode (**`--verbose`**)

Specify **`--verbose`** to print all statements through entire routine.

### 3. Optional controls for spatiotemporal subsetting
<a id='overview_3'></a>

#### Geographic bounding box (**`--bounding_box BOUNDING_BOX`**)

An area of interest may be specified as `SNWE` coordinates using the **`--bounding_box`** option. Coordinates should be specified as a space delimited string surrounded by quotes. The common intersection between the user-specified spatial bounds and the spatial bounds computed from the station locations in the input file is then passed. This example below would restrict the analysis to stations over California:
**`--bounding_box '32 42 -125 -114'`**

If no area of interest is specified, by default the maximum spatial bounds computed based off of the coordinates in the input file is passed.

#### Gridcell spacing (**`--spacing SPACING`**)

Specify degree spacing of grid-cells for statistical analyses through **`--spacing`**

By default grid-cell spacing is set to 1°. If the specified grid-cell spacing is not a multiple of the spatial bounds of the dataset, the grid-cell spacing again defaults back to  1°.

#### Subset in time (**`--timeinterval TIMEINTERVAL`**)

Define temporal bounds with **`--timeinterval TIMEINTERVAL`** by specifying earliest YYYY-MM-DD date followed by latest date YYYY-MM-DD. For example: **`--timeinterval 2018-01-01 2019-01-01`**

By default, bounds set to earliest and latest time found in input file.

#### Seasonal interval (**`--seasonalinterval SEASONALINTERVAL`**)

#### Observational error limit (**`--obs_errlimit OBS_ERRLIMIT`**)

Specify observation error threshold with **`--obs_errlimit OBS_ERRLIMIT`** to discard observations with large uncertainties, where units reflect specified input **`--unit`** (or default `m`).

### 4. Supported types of individual station scatter-plots
<a id='overview_4'></a>

#### Plot station distribution (**`--station_distribution`**)

Illustrate each individual station with black markers.

#### Plot mean tropospheric zenith delay by station (**`--station_delay_mean`**)

Illustrate the tropospheric zenith delay mean for each station with a **`hot`** colorbar.

#### Plot standard deviation of tropospheric zenith delay by station  (**`--station_delay_stdev`**)

Illustrate the tropospheric zenith delay standard deviation for each station with a **`hot`** colorbar.

#### Plot phase/amplitude of tropospheric zenith delay by station  (**`--station_seasonal_phase`**)

Illustrate the phase/amplitude of tropospheric zenith delay for each station with a **`hot`** colorbar.

### 5. Supported types of gridded station plots
<a id='overview_5'></a>

#### Plot gridded station heatmap (**`--grid_heatmap`**)

Illustrate heatmap of gridded station array with a **`hot`** colorbar.

#### Plot gridded mean tropospheric zenith delay (**`--grid_delay_mean`**)

Illustrate gridded mean of station-wise means with a **`hot`** colorbar.
Alternatively plot absolute gridded delay mean (i.e. measure biased by stations with higher temporal sampling) with the option `--grid_delay_absolute_mean`

#### Plot gridded median tropospheric zenith delay (**`--grid_delay_median`**)

Illustrate gridded mean of station-wise medians with a **`hot`** colorbar.
Alternatively plot absolute gridded delay median (i.e. measure biased by stations with higher temporal sampling) with the option `--grid_delay_absolute_median`

#### Plot gridded standard deviation of tropospheric zenith delay  (**`--grid_delay_stdev`**)

Illustrate gridded mean of station-wise standard deviations with a **`hot`** colorbar.
Alternatively plot absolute gridded delay standard deviation (i.e. measure biased by stations with higher temporal sampling) with the option `--grid_delay_absolute_stdev`

#### Plot gridded station-wise delay phase/amplitude  (**`--grid_seasonal_phase`**)

Illustrate gridded mean of station-wise phase/amplitude/period with a **`hot`** colorbar.
Alternatively plot absolute gridded delay phase/amplitude/period (i.e. measure biased by stations with higher temporal sampling) with the option `--grid_seasonal_absolute_phase`

### 6. Supported types of variogram plots
<a id='overview_6'></a>

#### Plot variogram (**`--variogramplot`**)

Passing **`--variogramplot`** toggles plotting of gridded station variogram, where gridded sill and range values for the experimental variogram fits are illustrated.

#### Apply experimental fit to binned variogram (**`--binnedvariogram`**)

Pass **`--binnedvariogram`** to apply experimental variogram fit to total binned empirical variograms for each time slice. 

Default is to pass total unbinned empiricial variogram.

#### Save variogram figures per time-slice (**`--variogram_per_timeslice`**)

Specify **`--variogram_per_timeslice`** to generate variogram plots per gridded station AND time-slice.

If option not toggled, then variogram plots are only generated per gridded station and spanning entire time-span.

#### Variogram error limit (**`--variogram_errlimit VARIOGRAM_ERRLIMIT`**)

Specify variogram RMSE threshold with **`--variogram_errlimit VARIOGRAM_ERRLIMIT`** to discard grid-cells with large RMSE.

### 7. Optional controls for plotting
<a id='overview_7'></a>

#### Save gridded array(s) as raster(s) (**`--grid_to_raster`**)

Save specified gridded array(s) as raster(s). 
May directly load/plot in successive script call by passing output grid as argument for **`--file`**.
E.g. if specified with **`--grid_delay_mean`**, then a raster file named `ZTD_grid_delay_mean.tif` containing the gridded mean delay will be generated.

#### Save debug figures of station-wise seasonal fit (**`--phaseamp_per_station`**)

Save debug figures of curve-fit vs data per station for seasonal amplitude/phase analaysis options (i.e. **`--grid_seasonal_phase`** and/or **`--station_seasonal_phase`**).

#### Minimum TS span and minimum fractional observations for seasonal amplitude/phase analyses (**`--min_span`**)

Minimum TS span (years) and minimum fractional observations in span (fraction) imposed for seasonal amplitude/phase analyses to be performed for a given station.
By default set to 2 years and 0.6 respectively (i.e. **`--min_span 2 0.6`**)

#### Period limit for seasonal amplitude/phase analyses (**`--period_limit`**)

Period limit (years) imposed for seasonal amplitude/phase analyses to be performed for a given station.

#### Variogram density threshold (**`--densitythreshold DENSITYTHRESHOLD`**)

For variogram plots, a given grid-cell is only valid if it contains this specified threshold of stations. 

By default set to 10 stations.

#### Figure DPI (**`--figdpi FIGDPI`**)

DPI to use for saving figures.

#### Plot title (**`--user_title USER_TITLE`**)

Specify custom title for plots.

#### Plot format (**`--plot_format PLOT_FMT`**)

File format for saving plots. Default is PNG.

#### Colorbar bounds (**`--color_bounds CBOUNDS`**)

Set lower and upper-bounds for plot colorbars. For example: **`--color_bounds '0 100'`**

By default set to the dynamic range of the data.

#### Colorbar percentile limits (**`--colorpercentile COLORPERCENTILE COLORPERCENTILE`**)

Set lower and upper percentile for plot colorbars. For example: **`--colorpercentile 30 100`**

By default set to 25% and 95%, respectively.

#### Superimpose individual stations over gridded array (**`--stationsongrids`**)

In gridded plots, superimpose your gridded array with a scatterplot of station locations.

#### Draw gridlines (**`--drawgridlines`**)

In gridded plots, draw gridlines.

#### Generate all supported plots (**`--plotall`**)

Generate all supported plots, as outlined under sections #4, #5, and #6 above.

## Examples of the **`raiderStats.py`** program
<a id='examples'></a>

### Example 1. Generate all individual station scatter-plots, as listed under [section #4](#overview_4) <a id='example_1'></a>

Using the file **`UNRcombinedGPS_ztd.csv`** generated by **`raiderDownloadGNSS.py`** as input, produce plots illustrating station distribution, mean tropospheric zenith delay by station, and standard deviation of tropospheric zenith delay by station.

Restrict the temporal span of the analyses to all data acquired between 2016-01-01 and 2018-12-31, and restrict the spatial extent to a geographic bounding box '32 42 -125 -114' spanning over California.
**NOTE** if not specified, by default the temporal bounds are set to earliest and latest time found in input file, and by default the maximum spatial bounds computed based off of the coordinates in the input file is passed.

The footprint of the specified geographic bounding box is depicted in **Fig. 1**.

These basic spatiotemporal constraints will be inherited by all successive examples.

In [36]:
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex1 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2018-12-31' --station_distribution --station_delay_mean --station_delay_stdev --cpus all

***Stats Function:***
- Plot spatial distribution of stations.
- Plot mean delay for each station.
- Plot delay stdev for each station.


Now we can take a look at the generated products:

In [20]:
!ls maps_ex1/figures

ZTD_station_delay_mean.png   ZTD_station_distribution.png
ZTD_station_delay_stdev.png


Here we visualize the spatial distribution of stations (*ZTD_station_distribution.png*) as blue markers.

![ZTD_station_distribution](support_docs/maps/maps_ex1/figures/ZTD_station_distribution.png?modified=00000000)

To generate this figure alone, run:
```
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex1 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2018-12-31' --station_distribution
```

Below is the mean tropospheric zenith delay by station (*ZTD_station_delay_mean.png*) with a **`hot`** colorbar. 

![ZTD_station_delay_mean](support_docs/maps/maps_ex1/figures/ZTD_station_delay_mean.png?modified=11111111)

To generate this figure alone, run:
```
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex1 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2018-12-31' --station_delay_mean
```

Below is the standard deviation of tropospheric zenith delay by station (*ZTD_station_delay_stdev.png*).

![ZTD_station_delay_stdev](support_docs/maps/maps_ex1/figures/ZTD_station_delay_stdev.png?modified=22222222)

To generate this figure alone, run:
```
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex1 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2018-12-31' --station_delay_stdev
```

### Example 2. Generate all basic gridded station plots, as listed under [section #5](#overview_5)<a id='example_2'></a>

Produce plots illustrating gridded station distribution, gridded mean tropospheric zenith delay, and gridded standard deviation of tropospheric zenith delay.

Also save gridded arrays as raster files with **`--grid_to_raster`** so as to more conveniently replot in successive script calls (recommended).

Finally, use the maximum number of allowed CPUs to save some time.

In [43]:
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex2 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2018-12-31' --grid_heatmap --grid_delay_mean --grid_delay_stdev --grid_to_raster --cpus all

***Stats Function:***
/Users/ssangha/Downloads/snap_setup/stable_oct5_2021/envs/RAiDER/lib/python3.10/site-packages/numpy/core/_asarray.py:130: RuntimeWarning: invalid value encountered in cast
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)
- Plot density of stations per gridcell.
- Plot mean of station-wise mean delay across each gridcell.
- Plot mean of station-wise stdev delay across each gridcell.


Now we can take a look at the generated rasters (i.e. the TIF files in the specified output directory):

In [ ]:
!ls maps_ex2

Now we can take a look at the generated plots:

In [ ]:
!ls maps_ex2/figures

Below is the heatmap of gridded station array (*ZTD_grid_heatmap.png*) with a **`hot`** colorbar.

Note that the colorbar bounds are somewhat saturated, which demonstrates the utility of plotting options outlined under section #7 such as **`--color_bounds`** and **`--colorpercentile`**

![ZTD_grid_heatmap](support_docs/maps/maps_ex2/figures/ZTD_grid_heatmap.png?modified=44444444)

To generate this figure alone, run:
```
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex2 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2018-12-31' --grid_heatmap --grid_to_raster
```

Below is the gridded mean tropospheric zenith delay (*ZTD_grid_delay_mean.png*).

![ZTD_grid_delay_mean](support_docs/maps/maps_ex2/figures/ZTD_grid_delay_mean.png?modified=33333333)

To generate this figure alone, run:
```
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex2 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2018-12-31' --grid_delay_mean --grid_to_raster
```

Below is the gridded standard deviation of tropospheric zenith delay (*ZTD_grid_delay_stdev.png*).

![ZTD_grid_delay_stdev](support_docs/maps/maps_ex2/figures/ZTD_grid_delay_stdev.png?modified=55555555)

To generate this figure alone, run:
```
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex2 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2018-12-31' --grid_delay_stdev --grid_to_raster
```

#### Example 2a. Redo plots efficiently using generated grid raster files <a id='example_2a'></a>

Since we haved the saved gridded arrays as raster files by specifying the **`--grid_to_raster`** option, we may directly replot these graphics by passing a given output raster file as input (e.g. `--file ZTD_grid_heatmap.tif`).

This is a practical, efficient means to adjust/replot graphics and save a great deal of time by avoiding the gridding/prep steps involved with processing the initial input CSV file, especially for cases which span continental/global scales.

Note though that since the output rasters are static with respect to the original specified spatiotemporal constraints (e.g. `--bounding_box` and `--timeinterval`), you cannot adjust such options with the rasters as input arguments. These rasters must be re-computed for any adjusted spatiotemporal parameters (if necessary) before replotting.

For this replotting command, let us also adjust the colorbar-bounds (using the `--color_bounds` option).

In [47]:
!raiderStats.py --file maps_ex2/ZTD_grid_heatmap.tif --workdir maps_ex2a --color_bounds '10 40'

***Stats Function:***
metadata_dict {'AREA_OR_POINT': 'Area', 'colorbarfmt': '%1i', 'gridfile_type': 'grid_heatmap', 'plotbbox': '-125.0 -114.0 32.0 42.0', 'spacing': '1.0', 'stationsongrids': 'False', 'time_lines': 'False', 'unit': 'm'}
- Plot density of stations per gridcell.


Below is the replotted heatmap of gridded station array (*ZTD_grid_heatmap.png*).

![ZTD_grid_heatmap](support_docs/maps/maps_ex2a/figures/ZTD_grid_heatmap.png?modified=1)

### Example 3. Generate gridded mean tropospheric zenith delay plot, with stations superimposed <a id='example_3'></a>

Produce plot illustrating gridded mean tropospheric zenith delay, superimposed with individual station locations (`--stationsongrids`).

Additionally, subset data in time for spring. I.e. **`'03-21 06-21'`**

Finally, use the maximum number of allowed CPUs to save some time.

In [10]:
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex3 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2018-12-31' --seasonalinterval '03-21 06-21' --grid_delay_mean --stationsongrids --cpus all

***Stats Function:***
- Plot mean of station-wise mean delay across each gridcell.


Now we can take a look at the generated plot:

In [11]:
!ls maps_ex3/figures

ZTD_grid_delay_mean.png


Below is the gridded mean tropospheric zenith delay (*ZTD_grid_delay_mean.png*) with a **`hot`** colorbar, with superimposed station locations denoted by blue markers.

![ZTD_grid_delay_mean](support_docs/maps/maps_ex3/figures/ZTD_grid_delay_mean.png?modified=2)

### Example 4. Generate variogram plots, as listed under [section #6](#overview_6)<a id='example_4'></a>

#### Variogram analysis overview

A variogram is the characterization of the difference between field values at two locations across a specified period of time `--timeinterval` and/or annual interval `--seasonalinterval`.

Variogram analysis through this program is performed for each grid-cell containing at least a specified threshold of stations `--densitythreshold` (by default 10 stations) and with <=1000 randomly selected pairs (*hardcoded* as `Nsamp=1000`). Before variogram fitting, the data across each grid-cell is detrended using a linear ramp derived from a least-squares solution.

The empirical variogram, or semivariance, provides a description of how the data are correlated with distance and is derived as half of the square difference between two observations:
1/2 * sqrt(obs1 - obs2)

The empirical variogram is then binned into increments of 20, with the top 3rd percentile discarded. These parameters are *hardcoded*.

Finally, the experimental variogram is derived using the binned variogram above as input and utilizing an exponential model fit by default (*hardcoded*). The experimental variogram practically is a discrete function calculated using a measure of variability between pairs of points at various distances. **Note** toggling the `--binnedvariogram` option would lead to the an experimental variogram fit to every binned time-slice for a given grid-cell, whereas the default is to bin all data across all time-slices and then fit the experimental variogram for a given grid-cell

From this procedure, we derive and plot the output sill/variance and range values. Respectively, the sill/variance represents the limit of the variogram representing where the variogram appears to level off. And the range represents the distance in which the difference of the variogram from the sill becomes negligible, and should not exceed half the size of a given grid-cell.

Produce plots illustrating empirical/experimental variogram fits per gridded station and time-slice (**`--variogram_per_timeslice`**) and also spanning the entire time-span. Plots of gridded station experimental variogram-derived sill and range values also generated.

Additionally, subset data in time for spring. I.e. **`'03-21 06-21'`**

Also save gridded arrays as raster files with **`--grid_to_raster`** so as to more conveniently replot in successive script calls (recommended).

Finally, use the maximum number of allowed CPUs to save some time.

In [58]:
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex4 --bounding_box '32 42 -125 -114' --timeinterval '2016-01-01 2019-12-31' --seasonalinterval '03-21 06-21' --variogramplot --variogram_per_timeslice --grid_to_raster --cpus all

***Stats Function:***
pre Date 2016-01-01 00:00:00 2018-12-31 00:00:00
/Users/ssangha/Downloads/snap_setup/stable_oct5_2021/RAiDER/tools/RAiDER/cli/statsPlot.py:976: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.df = filtered_self.append(filtered_self_ly, ignore_index=True)
post Date 2016-03-21 00:00:00 2018-06-19 00:00:00
***Variogram Analysis Function:***
- Plot variogram range per gridcell.
- Plot variogram sill per gridcell.
- Plot variogram RMSE per gridcell.


Now we can take a look at the generated variograms:

In [ ]:
!ls maps_ex4/variograms

There are several subdirectories corresponding to each grid-cell that each contain empirical and experimental variograms generated for each time-slice (e.g. **`grid34_timeslice20160321_justEMPvariogram.eps`** and **`grid34_timeslice20160321_justEXPvariogram.eps`**, respectively) and across the entire sampled time period (**`grid34_timeslice20160321–20180619_justEMPvariogram.eps`** and **`grid34_timeslice20160321–20180619_justEXPvariogram.eps`**, respectively).

Recall that the former pair of empirical/experimental variograms per time-slice are generated only if the **`---variogram_per_timeslice`** option is toggled. By default only the latter two pair of empirical/experimental variograms spanning the entire time-span are generated.

Here we visualize the total empirical variogram corresponding to the entire sampled time period for grid-cell 34 in the array (*grid34_timeslice20160321–20180619_justEMPvariogram.eps*). 

![justEMPvariogram](support_docs/maps/maps_ex4/variograms/grid34/grid34_timeslice20160321–20180619_justEMPvariogram.png?modified=3)

Here we visualize the total experimental variogram corresponding to the entire sampled time period for grid-cell 34 in the array (*grid34_timeslice20160321–20180619_justEXPvariogram.eps*). 

![justEXPvariogram](support_docs/maps/maps_ex4/variograms/grid34/grid34_timeslice20160321–20180619_justEXPvariogram.png?modified=4)

The central coordinates for all grid-nodes that satisfy the specified station density threshold (**`--densitythreshold`**, by default 10 stations per grid-cell) for variogram plots are stored in a lookup table:

In [12]:
!head maps_ex4/variograms/gridlocation_lookup.txt

grid1 Lat:40.5 Lon:-124.5
grid11 Lat:40.5 Lon:-123.5
grid12 Lat:39.5 Lon:-123.5
grid20 Lat:41.5 Lon:-122.5
grid22 Lat:39.5 Lon:-122.5
grid23 Lat:38.5 Lon:-122.5
grid24 Lat:37.5 Lon:-122.5
grid31 Lat:40.5 Lon:-121.5
grid33 Lat:38.5 Lon:-121.5
grid34 Lat:37.5 Lon:-121.5


Now we can take a look at the other generated figures:

In [60]:
!ls maps_ex4/figures

ZTD_grid_range.png          ZTD_grid_variogram_rmse.png
ZTD_grid_variance.png


Below is the gridded experimental variogram range (*ZTD_grid_range.png*) with a **`hot`** colorbar.

![ZTD_grid_range](support_docs/maps/maps_ex4/figures/ZTD_grid_range.png?modified=6)

Below is the gridded experimental variogram variance (*ZTD_grid_variance.png*).

![ZTD_grid_variance](support_docs/maps/maps_ex4/figures/ZTD_grid_variance.png?modified=6)

#### Examine extreme range of variogram fits

Below are a couple case examples of total experimental variograms corresponding to grid-cells which are associated with extreme range values. This may demonstrate the limitations of this approach and/or input dataset.

Here we visualize the total experimental variogram corresponding to the entire sampled time period for grid-cell 12 in the array (*grid12_timeslice20160321–20180619_justEXPvariogram.eps*). 

![justEXPvariogram](support_docs/maps/maps_ex4/variograms/grid12/grid12_timeslice20160321–20180619_justEXPvariogram.png?modified=7)

Here we visualize the total experimental variogram corresponding to the entire sampled time period for grid-cell 89 in the array (*grid89_timeslice20160321–20180619_justEXPvariogram.eps*). 

![justEXPvariogram2](support_docs/maps/maps_ex4/variograms/grid89/grid89_timeslice20160321–20180619_justEXPvariogram.png?modified=8)

Below is the gridded experimental variogram range labelled with grid-cells of select variograms from above (*ZTD_grid_range_labelled.png*).

![ZTD_grid_labelled_range](support_docs/ZTD_grid_range_labelled.png?modified=9)

### Example 5. Generate seasonal phase/amplitude plots, as listed under [section #5](#overview_5)<a id='example_5'></a>

#### Seasonal phase/amplitude analysis overview

The following sinusoidal form is used in this program to derive seasonal metrics:

**A * sin(ωt + p) + c**


Where *A* = amplitude (m), *ω* = angular frequency (rad/s), *p* = phase shift (days, positive to the left of origin), *c* = offset (days)

And from which we derive and pass frequency (1/years) = (*ω*/2π) * 31556952, and period = 1/frequency (scaled to years)

Finally the conventional origin *t0* = January 1, 1970

<img src="support_docs/Sinusoidal_form.png" alt="footprint" width="700">
<center><b>Fig. 2</b> Sinusoidal form used to derive seasonal metrics.</center>

Produce plots illustrating seasonal phase/amplitude/period fits for each individual station (**`--station_seasonal_phase`**) and averaged across each grid-cell (**`--grid_seasonal_phase`**). The standard deviation is also plotted across each grid-cell.

Control the prerequisite minimum time-series span (years) a given station TS must span, and the prerequisite minimum fractional observations in span (fraction across specified `--timeinterval TIMEINTERVAL`, by default the entire span of input dataset). Here, we will specify a minimum time-series span of 2 years, and minimum fraction observation of 0.1 (i.e. **`--min_span 2 0.1`**). **Note** the latter constraint is intentionally loose as we have only downloaded data for every 5 days of the years, such that 5/365 ~ 0.014 would be expected for 'full' coverage through the specified time interval. Thus, the denser the temporal sampling, the more conservative you can be with the fractional observation control.

Save figures of curve-fits vs data per station with  **`--phaseamp_per_station`** for debugging purposes. Not recommended for large-scale runs in the interest of speed/efficiency.

Also save gridded arrays as raster files with **`--grid_to_raster`** so as to more conveniently replot in successive script calls (recommended).

Finally, use the maximum number of allowed CPUs to save some time.

In [6]:
!raiderStats.py --file products/UNRcombinedGPS_ztd.csv --workdir maps_ex5 --bounding_box '32 42 -125 -114' --grid_seasonal_phase --station_seasonal_phase --min_span 2 0.1 --phaseamp_per_station --grid_to_raster --cpus all

***Stats Function:***
- Plot delay phase/amplitude for each station.
- Plot mean of station-wise delay phase across each gridcell.
- Plot mean of station-wise delay amplitude across each gridcell.
- Plot mean of station-wise delay period across each gridcell.
- Plot mean stdev of station-wise delay phase across each gridcell.
- Plot mean stdev of station-wise delay amplitude across each gridcell.
- Plot mean stdev of station-wise delay period across each gridcell.
- Plot mean of seasonal fit RMSE across each gridcell.


Now we can take a look at the generated rasters (i.e. the TIF files in the specified output directory):

In [7]:
!ls maps_ex5

ZTD_grid_seasonal_amplitude.tif       ZTD_grid_seasonal_phase.tif
ZTD_grid_seasonal_amplitude_stdev.tif ZTD_grid_seasonal_phase_stdev.tif
ZTD_grid_seasonal_fit_rmse.tif        figures
ZTD_grid_seasonal_period.tif          phaseamp_per_station
ZTD_grid_seasonal_period_stdev.tif


Now we can take a look at the generated debug figures illustrating the curve-fits vs data (**`--phaseamp_per_station`**):

In [8]:
!ls maps_ex5/phaseamp_per_station

station7ODM.png stationFPPB.png stationP134.png stationP473.png stationP790.png
stationACSB.png stationFSHB.png stationP135.png stationP474.png stationP793.png
stationACSX.png stationFVPK.png stationP136.png stationP475.png stationP794.png
stationAERO.png stationFZHS.png stationP137.png stationP476.png stationP795.png
stationAGMT.png stationGARL.png stationP138.png stationP477.png stationP796.png
stationAIAH.png stationGERL.png stationP139.png stationP478.png stationP797.png
stationALPP.png stationGHRP.png stationP140.png stationP479.png stationP799.png
stationALTH.png stationGISA.png stationP141.png stationP480.png stationP800.png
stationANA1.png stationGLRS.png stationP142.png stationP481.png stationP808.png
stationAR27.png stationGMAG.png stationP143.png stationP482.png stationP809.png
stationAR53.png stationGMPK.png stationP144.png stationP483.png stationP810.png
stationARM1.png stationGMRC.png stationP145.png stationP484.png stationP811.png
stationARM2.png stationGNPS.png stationP

Here we visualize the time-series and curve-fit corresponding for one of the stations.

![stationCACH](support_docs/maps/maps_ex5/phaseamp_per_station/stationP335.png?modified=10)

In [9]:
!ls maps_ex5/figures

ZTD_grid_seasonal_amplitude.png       ZTD_grid_seasonal_phase.png
ZTD_grid_seasonal_amplitude_stdev.png ZTD_grid_seasonal_phase_stdev.png
ZTD_grid_seasonal_fit_rmse.png        ZTD_station_delay_period.png
ZTD_grid_seasonal_period.png          ZTD_station_seasonal_amplitude.png
ZTD_grid_seasonal_period_stdev.png    ZTD_station_seasonal_phase.png


Now we can take a look at the generated plots:

Below is the seasonal phase of tropospheric zenith delay by station (*ZTD_station_seasonal_phase.png*) with a **`hot`** colorbar.

![ZTD_station_seasonal_phase](support_docs/maps/maps_ex5/figures/ZTD_station_seasonal_phase.png?modified=11)

Below is the gridded mean of the station-wise seasonal phase of tropospheric zenith delay (*ZTD_grid_seasonal_phase.png*).

![ZTD_grid_seasonal_phase](support_docs/maps/maps_ex5/figures/ZTD_grid_seasonal_phase.png?modified=11)

Below is the seasonal amplitude of tropospheric zenith delay by station (*ZTD_station_seasonal_amplitude.png*).

![ZTD_station_seasonal_amplitude](support_docs/maps/maps_ex5/figures/ZTD_station_seasonal_amplitude.png?modified=12)

Below is the gridded mean of the station-wise seasonal amplitude of tropospheric zenith delay (*ZTD_grid_seasonal_amplitude.png*).

![ZTD_grid_seasonal_amplitude](support_docs/maps/maps_ex5/figures/ZTD_grid_seasonal_amplitude.png?modified=13)

Below is the seasonal period of tropospheric zenith delay by station (*ZTD_station_delay_period.png*).

![ZTD_station_delay_period](support_docs/maps/maps_ex5/figures/ZTD_station_delay_period.png?modified=14)

Below is the gridded period of the station-wise seasonal period of tropospheric zenith delay (*ZTD_grid_seasonal_period.png*).

![ZTD_grid_seasonal_period](support_docs/maps/maps_ex5/figures/ZTD_grid_seasonal_period.png?modified=15)

Below is the gridded period standard deviation of the station-wise seasonal phase of tropospheric zenith delay (*ZTD_grid_seasonal_period_stdev.png*).

![ZTD_grid_seasonal_period_stdev](support_docs/maps/maps_ex5/figures/ZTD_grid_seasonal_period_stdev.png?modified=16)

### Example 6. Generate weather model/GNSS residual plots <a id='example_6'></a>

Produce plots illustrating the residual between tropospheric zenith delay at specified GNSS stations and collocated weather-model delay nodes.

GNSS data will again be downloaded with **`raiderDownloadGNSS.py`**, and GMAO weather-model derived delay will be computed with **`raiderDelay.py`**

Virtually access GNSS station location and zenith delay information for the year '2019', for every 12 days, at a UTC time of day 'HH:MM:SS' of '00:00:00', and across a geographic bounding box '32 42 -125 -114' spanning over California.

The footprint of the specified geographic bounding box is again depicted in **Fig. 1**.

In addition to querying for multiple years, we will also experiment with using the maximum number of allowed CPUs to save some time! Recall again that the default number of CPUs used for parallelization is 8.

Note these features and similar examples are outlined in more detail in the companion notebook **`raiderDownloadGNSS/raiderDownloadGNSS_tutorial.ipynb`**.

**NOTE** the following two sections will take ~12 minutes to complete. It is *recommended* to take a coffee break!

In [62]:
!raiderDownloadGNSS.py --out GNSS_2019 --date 20190101 20191231 12 --returntime '00:00:00' --bounding_box '32 42 -125 -114' --cpus all

1781 stations were found in [32.0, 42.0, 235.0, 246.0]
Currently checking station 34A1 in 2019
Currently checking station BAR1 in 2019
Currently checking station BRDY in 2019
Currently checking station CAEU in 2019
Currently checking station CASA in 2019
Currently checking station CLIO in 2019
Currently checking station DAM1 in 2019
Currently checking station ELDO in 2019
Currently checking station ELGI in 2019
Currently checking station CLOV in 2019
Currently checking station DAM2 in 2019
Currently checking station CASD in 2019
Currently checking station BRI2 in 2019
Currently checking station 34A2 in 2019
Currently checking station CAFA in 2019
Currently checking station BATM in 2019
Currently checking station BRIB in 2019
Currently checking station 34A3 in 2019
Currently checking station DAM3 in 2019
Currently checking station ELKG in 2019
Currently checking station BATT in 2019
Currently checking station CAFL in 2019
Currently checking station CLV1 in 2019
Currently checking statio

Compute tropospheric zenith delay from the GMAO weather-model for the year '2019', for every 12 days, at a UTC time of day 'HH:MM:SS' of '00:00:00', at stations located across a geographic bounding box '32 42 -125 -114' spanning over California and captured in the `GNSS_2019/gnssStationList_overbbox.csv` list generated by the `raiderDownloadGNSS.py` above (i.e. `--station_file GNSS_2019/gnssStationList_overbbox.csv`), and with an integration height limit `--zref` of 30,000 m.

*All of these parameters have been set beforehand within the provided `raider.yaml` file.* Inspect manually to verify the input fields.

The footprint of the specified geographic bounding box is again depicted in **Fig. 1**.

In [2]:
!mkdir GMAO_2019

In [3]:
os.chdir('GMAO_2019')

In [5]:
!ls ../support_docs/raider.yaml

../support_docs/raider.yaml


In [6]:
!raider.py ../support_docs/raider.yaml

Starting to run the weather model calculation
Date: 20190101
Beginning weather model pre-processing
Weather model GMAO is available from 2014-02-20 to Present
Number of weather model nodes: 286230
Shape of weather model: (47, 42, 145)
Bounds of the weather model: 31.00/42.50/-125.62/-112.81 (SNWE)
Weather model: GMAO
Mean value of the wet refractivity: 5.396592
Mean value of the hydrostatic refractivity: 104.953026

======Weather Model class object=====
Weather model time: 2019-01-01 00:00:00
Latitude resolution: 0.25
Longitude resolution: 0.3125
Native projection: EPSG:4326
ZMIN: -100.0
ZMAX: 15000.0
k1 = 0.776
k2 = 0.233
k3 = 3750.0
Humidity type = q
Class name: gmao
Dataset: gmao
A: []
B: []
Number of points in Lon/Lat = 47/42
Total number of grid points (3D): 286230

Output SNWE: [31.5, 42.46875, -124.875, -113.625]
Output cube spacing: 0.28125
Wrote delays to: ./GMAO_Delay_20190101T000000.csv
Starting to run the weather model calculation
Date: 20190113
Beginning weather model pre-

In [7]:
os.chdir('../')

Combine delay files derived above from the GMAO weather-model (`--raider 'GMAO_2019/GMAO_Delay_*.csv' --raiderDir GMAO_2019 --raider_column totalDelay`) and GNSS stations (`GNSS_2019/UNRcombinedGPS_ztd.csv --column ZTD`) respectively, passing only data which are collocated in space and time.

In [12]:
!raiderCombine.py --gnss GNSS_2019/UNRcombinedGPS_ztd.csv --column ZTD --raider 'GMAO_2019/GMAO_Delay_*.csv' --raiderDir GMAO_2019 --raider_column totalDelay --out Combined_delays_GNSSandGMAO_2019.csv

Merging delay files GMAO_2019/GMAO_Delay_*.csv and GNSS_2019/UNRcombinedGPS_ztd.csv
Beginning merge
Total number of rows in the concatenated file: 36487
Total number of rows containing NaNs: 4320
Merge finished


Using the file **`Combined_delays_GNSSandGMAO_2019.csv`** generated by **`raiderCombine.py`** as input and passing the weather-model/GNSS residual values (`--column_name ZTD_minus_RAiDER`), produce plots illustrating mean tropospheric zenith delay by station + across each grid-cell, and standard deviation of tropospheric zenith delay by station + across each grid-cell.

In [13]:
!raiderStats.py --file Combined_delays_GNSSandGMAO_2019.csv --column_name ZTD_minus_RAiDER --workdir maps_ex6 --bounding_box '32 42 -125 -114' --station_delay_mean --station_delay_stdev --grid_delay_mean --grid_delay_stdev --grid_to_raster --cpus all

***Stats Function:***
- Plot mean delay for each station.
- Plot delay stdev for each station.
- Plot mean of station-wise mean delay across each gridcell.
- Plot mean of station-wise stdev delay across each gridcell.


Below is the mean tropospheric zenith delay by station (*ZTD_minus_RAiDER_station_delay_mean.png*) with a **`hot`** colorbar. 

<img src="support_docs/maps/maps_ex6/figures/ZTD_minus_RAiDER_station_delay_mean.png" alt="ZTD_minus_RAiDER_station_delay_mean" width="700">

To generate this figure alone, run:
```
!raiderStats.py --file Combined_delays_GNSSandGMAO_2019.csv --column_name ZTD_minus_RAiDER  --workdir maps_ex6 --bounding_box '32 42 -125 -114' --timeinterval '2019-01-01 2019-12-31' --station_delay_mean
```

Below is the standard deviation of tropospheric zenith delay by station (*ZTD_minus_RAiDER_station_delay_stdev.png*).  

<img src="support_docs/maps/maps_ex6/figures/ZTD_minus_RAiDER_station_delay_stdev.png" alt="ZTD_minus_RAiDER_station_delay_stdev" width="700">

To generate this figure alone, run:
```
!raiderStats.py --file Combined_delays_GNSSandGMAO_2019.csv --column_name ZTD_minus_RAiDER  --workdir maps_ex6 --bounding_box '32 42 -125 -114' --timeinterval '2019-01-01 2019-12-31' --station_delay_stdev
```

Below is the gridded mean tropospheric zenith delay (*ZTD_minus_RAiDER_grid_delay_mean.png*).

<img src="support_docs/maps/maps_ex6/figures/ZTD_minus_RAiDER_grid_delay_mean.png" alt="ZTD_minus_RAiDER_grid_delay_mean" width="700">

To generate this figure alone, run:
```
!raiderStats.py --file Combined_delays_GNSSandGMAO_2019.csv --column_name ZTD_minus_RAiDER  --workdir maps_ex6 --bounding_box '32 42 -125 -114' --timeinterval '2019-01-01 2019-12-31' --grid_delay_mean
```

Below is the gridded standard deviation of tropospheric zenith delay (*ZTD_minus_RAiDER_grid_delay_stdev.png*).

<img src="support_docs/maps/maps_ex6/figures/ZTD_minus_RAiDER_grid_delay_stdev.png" alt="ZTD_minus_RAiDER_grid_delay_stdev" width="700">

To generate this figure alone, run:
```
!raiderStats.py --file Combined_delays_GNSSandGMAO_2019.csv --column_name ZTD_minus_RAiDER  --workdir maps_ex6 --bounding_box '32 42 -125 -114' --timeinterval '2019-01-01 2019-12-31' --grid_delay_stdev
```